In [1]:
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import os
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import h5py
import numpy as np
import os

Using TensorFlow backend.


In [4]:
data_path = './ModelNet40/'

for d in [['train', len(os.listdir(data_path + 'train'))], ['test', len(os.listdir(data_path + 'test'))]]:
    data = None
    labels = None
    for j in range(d[1]):
        file_name = data_path + d[0] + '/ply_data_{0}{1}.h5'.format(d[0], j)
        f = h5py.File(file_name, mode='r')
        if data is None:
            data = f['data']
            labels = f['label']
        else:
            data = np.vstack((data, f['data']))
            labels = np.vstack((labels, f['label']))
    f.close()
    save_name = data_path + '/ply_data_{0}.h5'.format(d[0])
    print(data.shape)
    print(labels.shape)
    h5_fout = h5py.File(save_name)
    h5_fout.create_dataset(
        'data', data=data,
        dtype='float32')
    h5_fout.create_dataset(
        'label', data=labels,
        dtype='float32')
    h5_fout.close()


(9840, 2048, 3)
(9840, 1)


ValueError: Unable to create dataset (no write intent on file)

In [94]:
nb_classes = 40
train_file = './ModelNet40/train/ply_data_train0.h5'
test_file = './ModelNet40/test/ply_data_test1.h5'

In [95]:
epochs = 100
batch_size = 32

In [96]:
from keras.utils import np_utils

In [97]:
import numpy as np
import h5py
import random
from keras.utils import np_utils
class DataGenerator:
    def __init__(self, file_name, batch_size, nb_classes=40, train=True):
        self.fie_name = file_name
        self.batch_size = batch_size
        self.nb_classes = nb_classes
        self.train = train

    @staticmethod
    def rotate_point_cloud(data):
        """ Randomly rotate the point clouds to augument the dataset
            rotation is per shape based along up direction
            Input:
              Nx3 array, original point clouds
            Return:
              Nx3 array, rotated point clouds
        """
        rotation_angle = np.random.uniform() * 2 * np.pi
        cosval = np.cos(rotation_angle)
        sinval = np.sin(rotation_angle)
        rotation_matrix = np.array([[cosval, 0, sinval],
                                    [0, 1, 0],
                                    [-sinval, 0, cosval]])
        rotated_data = np.dot(data.reshape((-1, 3)), rotation_matrix)
        return rotated_data

    @staticmethod
    def jitter_point_cloud(data, sigma=0.01, clip=0.05):
        """ Randomly jitter points. jittering is per point.
            Input:
              Nx3 array, original point clouds
            Return:
              Nx3 array, jittered point clouds
        """
        N, C = data.shape
        assert (clip > 0)
        jittered_data = np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
        jittered_data += data
        return jittered_data

    def generator(self):
        f = h5py.File(self.fie_name, mode='r')
        nb_sample = f['data'].shape[0]
        while True:
            index = [n for n in range(nb_sample)]
            random.shuffle(index)
            for i in range(nb_sample // self.batch_size):
                batch_start = i * self.batch_size
                batch_end = (i + 1) * self.batch_size
                batch_index = index[batch_start: batch_end]
                X = []
                Y = []
                for j in batch_index:
                    item = f['data'][j]
                    label = f['label'][j]
                    if self.train:
                        is_rotate = random.randint(0, 1)
                        is_jitter = random.randint(0, 1)
                        if is_rotate == 1:
                            item = self.rotate_point_cloud(item)
                        if is_jitter == 1:
                            item = self.jitter_point_cloud(item)
                    X.append(item)
                    Y.append(label[0])
                Y = np_utils.to_categorical(np.array(Y), self.nb_classes)
                yield np.array(X), Y


In [98]:
train = DataGenerator(train_file, batch_size, nb_classes, train=True)
val = DataGenerator(test_file, batch_size, nb_classes, train=False)

In [99]:
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dropout, Input, BatchNormalization, Dense
from keras.layers import Reshape, Lambda, concatenate
from keras.models import Model
from keras.engine.topology import Layer
import numpy as np
import tensorflow as tf

In [100]:
class MatMul(Layer):

    def __init__(self, **kwargs):
        super(MatMul, self).__init__(**kwargs)

    def build(self, input_shape):
        # Used purely for shape validation.
        if not isinstance(input_shape, list):
            raise ValueError('`MatMul` layer should be called '
                             'on a list of inputs')
        if len(input_shape) != 2:
            raise ValueError('The input of `MatMul` layer should be a list containing 2 elements')

        if len(input_shape[0]) != 3 or len(input_shape[1]) != 3:
            raise ValueError('The dimensions of each element of inputs should be 3')

        if input_shape[0][-1] != input_shape[1][1]:
            raise ValueError('The last dimension of inputs[0] should match the dimension 1 of inputs[1]')

    def call(self, inputs):
        if not isinstance(inputs, list):
            raise ValueError('A `MatMul` layer should be called '
                             'on a list of inputs.')
        return tf.matmul(inputs[0], inputs[1])

    def compute_output_shape(self, input_shape):
        output_shape = [input_shape[0][0], input_shape[0][1], input_shape[1][-1]]
        return tuple(output_shape)


In [101]:
def PointNet(nb_classes):
    input_points = Input(shape=(2048, 3))
    # issues
    # input transformation net
    x = Conv1D(64, 1, activation='relu')(input_points)
    x = BatchNormalization()(x)
    x = Conv1D(128, 1, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Conv1D(1024, 1, activation='relu')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2048)(x)

    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)

    x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
    input_T = Reshape((3, 3))(x)

    # forward net
    g = MatMul()([input_points, input_T])
    g = Conv1D(64, 1, activation='relu')(g)
    g = BatchNormalization()(g)
    g = Conv1D(64, 1, activation='relu')(g)
    g = BatchNormalization()(g)

    # feature transform net
    f = Conv1D(64, 1, activation='relu')(g)
    f = BatchNormalization()(f)
    f = Conv1D(128, 1, activation='relu')(f)
    f = BatchNormalization()(f)
    f = Conv1D(1024, 1, activation='relu')(f)
    f = BatchNormalization()(f)
    f = MaxPooling1D(pool_size=2048)(f)
    f = Dense(512, activation='relu')(f)
    f = BatchNormalization()(f)
    f = Dense(256, activation='relu')(f)
    f = BatchNormalization()(f)
    f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
    feature_T = Reshape((64, 64))(f)

    # forward net
    g = MatMul()([g, feature_T])
    g = Conv1D(64, 1, activation='relu')(g)
    g = BatchNormalization()(g)
    g = Conv1D(128, 1, activation='relu')(g)
    g = BatchNormalization()(g)
    g = Conv1D(1024, 1, activation='relu')(g)
    g = BatchNormalization()(g)

    # global feature
    global_feature = MaxPooling1D(pool_size=2048)(g)

    # point_net_cls
    c = Dense(512, activation='relu')(global_feature)
    c = BatchNormalization()(c)
    c = Dropout(0.5)(c)
    c = Dense(256, activation='relu')(c)
    c = BatchNormalization()(c)
    c = Dropout(0.5)(c)
    c = Dense(nb_classes, activation='softmax')(c)
    prediction = Flatten()(c)

    model = Model(inputs=input_points, outputs=prediction)

    return model

In [102]:
model = PointNet(nb_classes)

In [103]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, 2048, 3)      0                                            
__________________________________________________________________________________________________
conv1d_78 (Conv1D)              (None, 2048, 64)     256         input_8[0][0]                    
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 2048, 64)     256         conv1d_78[0][0]                  
__________________________________________________________________________________________________
conv1d_79 (Conv1D)              (None, 2048, 128)    8320        batch_normalization_120[0][0]    
__________________________________________________________________________________________________
batch_norm

In [104]:
lr = 0.0001
adam = Adam(lr=lr)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [111]:
class MetaCheckpoint(ModelCheckpoint):
    """
    Checkpoints some training information with the model. This should enable
    resuming training and having training information on every checkpoint.
    Thanks to Roberto Estevao @robertomest - robertomest@poli.ufrj.br
    """

    def __init__(self, filepath, monitor='val_loss', verbose=0,
                 save_best_only=False, save_weights_only=False,
                 mode='auto', period=1, training_args=None, meta=None):

        super(MetaCheckpoint, self).__init__(filepath, monitor='val_loss',
                                             verbose=0, save_best_only=False,
                                             save_weights_only=False,
                                             mode='auto', period=1)

        self.filepath = filepath
        self.meta = meta or {'epochs': []}

        if training_args:
            self.meta['training_args'] = training_args

    def on_train_begin(self, logs={}):
        super(MetaCheckpoint, self).on_train_begin(logs)

    def on_epoch_end(self, epoch, logs={}):
        super(MetaCheckpoint, self).on_epoch_end(epoch, logs)

        # Get statistics
        self.meta['epochs'].append(epoch)
        for k, v in logs.items():
            # Get default gets the value or sets (and gets) the default value
            self.meta.setdefault(k, []).append(v)

        # Save to file
        filepath = self.filepath.format(epoch=epoch, **logs)

        if self.epochs_since_last_save == 0:
            with h5py.File(filepath, 'r+') as f:
                meta_group = f.create_group('meta')
                meta_group.attrs['training_args'] = yaml.dump(
                    self.meta.get('training_args', '{}'))
                meta_group.create_dataset('epochs',
                                          data=np.array(self.meta['epochs']))
                for k in logs:
                    meta_group.create_dataset(k, data=np.array(self.meta[k]))

In [112]:

if not os.path.exists('./results/'):
    os.mkdir('./results/')
checkpoint = MetaCheckpoint('./results/pointnet.h5', monitor='val_acc',
                             save_weights_only=True, save_best_only=True,
                             verbose=1)

In [113]:
import keras.backend as K
from keras.callbacks import Callback, ModelCheckpoint
import yaml

In [114]:
import keras.backend as K
from keras.callbacks import Callback
import yaml
import h5py
import numpy as np

class Step(Callback):

    def __init__(self, steps, learning_rates, verbose=0):
        self.steps = steps
        self.lr = learning_rates
        self.verbose = verbose

    def change_lr(self, new_lr):
        old_lr = K.get_value(self.model.optimizer.lr)
        K.set_value(self.model.optimizer.lr, new_lr)
        if self.verbose == 1:
            print('Learning rate is %g' %new_lr)

    def on_epoch_begin(self, epoch, logs={}):
        for i, step in enumerate(self.steps):
            if epoch < step:
                self.change_lr(self.lr[i])
                return
        self.change_lr(self.lr[i+1])

    def get_config(self):
        config = {'class': type(self).__name__,
                  'steps': self.steps,
                  'learning_rates': self.lr,
                  'verbose': self.verbose}
        return config

    @classmethod
    def from_config(cls, config):
        offset = config.get('epoch_offset', 0)
        steps = [step - offset for step in config['steps']]
        return cls(steps, config['learning_rates'],
                   verbose=config.get('verbose', 0))

In [115]:
def onetenth_50_75(lr):
    steps = [50, 75]
    lrs = [lr, lr / 10, lr / 100]
    return Step(steps, lrs)

In [116]:
history = model.fit_generator(train.generator(),
                              steps_per_epoch=2048 // batch_size,
                              epochs=epochs,
                              validation_data=val.generator(),
                              validation_steps=420 // batch_size,
                              callbacks=[checkpoint, onetenth_50_75(lr)],
                              verbose=1)

Epoch 1/100
64/64 [==============================] - 458s 7s/step - loss: 3.7932 - acc: 0.1743 - val_loss: 2.9075 - val_acc: 0.2620
Epoch 2/100
 1/64 [..............................] - ETA: 7:01 - loss: 3.9543 - acc: 0.1250

KeyboardInterrupt: 

In [123]:
import yaml
def load_meta(model_fname):
    ''' Load meta configuration
    '''
    meta = {}

    with h5py.File(model_fname, 'r') as f:
        meta_group = f['meta']

        meta['training_args'] = yaml.load(
            meta_group.attrs['training_args'])
        for k in meta_group.keys():
            meta[k] = list(meta_group[k])

    return meta

In [129]:
load_meta("./results/pointnet.h5")

{'acc': [0.26862785016286644,
  0.4483916938110749,
  0.5468241042345277,
  0.6088151465798045,
  0.6486156351791531,
  0.6781351791530945,
  0.7043973941368078,
  0.7199714983713354,
  0.7350366449511401,
  0.7538680781758957,
  0.7553949511400652,
  0.7729030944625407,
  0.780435667752443,
  0.7824714983713354,
  0.790207654723127,
  0.7892915309446255,
  0.8017100977198697,
  0.8089372964169381,
  0.8131107491856677,
  0.8189128664495114,
  0.815757328990228,
  0.8240024429967426,
  0.8209486970684039],
 'epochs': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22],
 'loss': [3.2427984048955216,
  2.206099193336909,
  1.7374054249800766,
  1.4651202674020773,
  1.2962882938136495,
  1.1659571400294475,
  1.0878957184790012,
  1.017467048832182,
  0.9487431292036845,
  0.882083866757368,
  0.868687592616687,
  0.8038371513060716,
  0.7782100742919438,
  0.7640379909972026,
  0.7306155241079362,
  0.73737922534

In [ ]:
def plot_history(history, result_dir):
    plt.plot(history.history['acc'], marker='.')
    plt.plot(history.history['val_acc'], marker='.')
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['acc', 'val_acc'], loc='lower right')
    plt.savefig(os.path.join(result_dir, 'model_accuracy.png'))
    plt.close()

    plt.plot(history.history['loss'], marker='.')
    plt.plot(history.history['val_loss'], marker='.')
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.grid()
    plt.legend(['loss', 'val_loss'], loc='upper right')
    plt.savefig(os.path.join(result_dir, 'model_loss.png'))
    plt.close()

In [ ]:
def save_history(history, result_dir):
    loss = history.history['loss']
    acc = history.history['acc']
    val_loss = history.history['val_loss']
    val_acc = history.history['val_acc']
    nb_epoch = len(acc)

    with open(os.path.join(result_dir, 'result.txt'), 'w') as fp:
        fp.write('epoch\tloss\tacc\tval_loss\tval_acc\n')
        for i in range(nb_epoch):
            fp.write('{}\t{}\t{}\t{}\t{}\n'.format(
                i, loss[i], acc[i], val_loss[i], val_acc[i]))
        fp.close()

In [ ]:
plot_history(history, './results/')
save_history(history, './results/')
model.save_weights('./results/pointnet_weights.h5')

In [70]:
import keras
from model_cls import PointNet
from data_loader import DataGenerator

nb_classes = 40
test_file = './ModelNet40/ply_data_test.h5'
epochs = 100
batch_size = 32
model = PointNet(nb_classes)
lr = 0.0001
adam = Adam(lr=lr)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
val = DataGenerator(test_file, batch_size, nb_classes, train=False)
model.load_weights("./results/pointnet.h5")

In [76]:
model.meta.get('epochs')

AttributeError: 'Model' object has no attribute 'meta'

In [15]:
model.load_weights()

AttributeError: 'Model' object has no attribute 'epoch'

In [34]:
text = "filename-{}-{}.h5"

In [35]:
epoch = 4

In [36]:
logs = {"arg3": 3, "arg2": "two", "arg1": 5}

In [37]:
text.format(epoch=epoch + 1, **logs)

IndexError: tuple index out of range

In [40]:
def load_h5(h5_filename):
    f = h5py.File(h5_filename)
    data = f['data'][:]
    label = f['label'][:]
    return (data, label)

In [91]:
t, l = load_h5("./ModelNet40/test/ply_data_test1.h5")

In [92]:
len(t)

420

In [93]:
len(l)

420